# Analysis Notebook for the scraper project

In [37]:
import pandas as pd
import os
import numpy as np
from pathlib import Path
import os
import sys
import re

In [39]:
pd.options.display.max_columns = None

In [40]:
root = Path(os.getcwd()).parent
fname = root / 'data' / 'scraper_df.csv'
df=pd.read_csv(fname)
df

,Unnamed: 0,alt,class,data-actions,data-advert-position,data-attr-href,data-category,data-label,data-trace,href,owner,timestamp,text,details,category1,bid_ask,rent_buy,rooms,cost,address,duration
0,Wunderschönes Co-Living Haus mit 38 Suiten in ...,Wunderschönes Co-Living Haus mit 38 Suiten in ...,[],click,2042131_1220_d49519d52efd3d369b3955fe61e2aafb,https://www.ronorp.net/zuerich/immobilien/wg-z...,ContentList,ContentTitle,NaN,https://www.ronorp.net/zuerich/immobilien/wg-z...,johannespeter,2020-02-09 10:53:00,"Ich glaube, dass zusammen einfach schöner ist ...",Biete / Suche / Tausche: Wohnung / Zimmer: Bie...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"Cozy Studio near Schmiede Wiedikon, 1405CHF/month","Cozy Studio near Schmiede Wiedikon, 1405CHF/month",[],click,2072688_1220_ff7c187b50787f7a14efb238f1d893c6,https://www.ronorp.net/zuerich/immobilien/wg-z...,ContentList,ContentTitle,NaN,https://www.ronorp.net/zuerich/immobilien/wg-z...,Songmin Zhao,2020-10-17 14:37:00,"Hi,I am moving out of my studio and would like...",www.newhome.ch Biete / Suche / Tausche: Wohnun...,Wohnung / Zimmer,Biete,Wohnung / Zimmer,1.0,1'405.00,8003 Zürich,unknown


In [15]:
df.loc[0,'details']

"Biete / Suche / Tausche: Wohnung / Zimmer: Biete Mieten & Kaufen: Mieten Stadt / Agglo: Agglo Vertragsart: unbefristet Zimmer: 1Kosten: 1'095.00Adresse: Marktgasse 19, 8038 Kloten Kontakt: i***@t*************hDie vollen Kontaktinformationen sind nur für Ron Orp Leser sichtbar. Hier gleich kostenlos anmelden. \t \t\t \t\tAchtung: Überweise nie eine Kaution an Personen, deren Wohnung du nie gesehen hast oder weil sie gerade nicht in der Schweiz weilen.\t \t mehr SCHREIB MIR! Anzeige Crowdfunding Workshop Food ZH Aufstrebende Gastronomen aufgepasst! Am Crowdify Workshop erhalten ambitionierte Feinschmecker den Masterplan für eine erfolgreiche Crowdfunding-Aktion. Mehr Infos Werbeanzeige WG Folgen ♥ ♥ ♥ Unterstütze Ron ♥ ♥ ♥Ich bin dein Sprachrohr für deine Stadt und dabei stets unabhängig. Daher freue ich mich über alle, die bereit sind, einen kleinen, finanziellen Beitrag an mein Tun zu leisten. Kommentare1 auf Merkliste"

In [14]:
df.loc[1,'details']

"www.newhome.ch Biete / Suche / Tausche: Wohnung / Zimmer: Biete Mieten & Kaufen: Mieten Stadt / Agglo: Stadt Zimmer: 1.0Kosten: 1'405.00Adresse: 8003 Zürich Kontakt:Die vollen Kontaktinformationen sind nur für Ron Orp Leser sichtbar. Hier gleich kostenlos anmelden. Original Inserat auf: \t \t\t \t\tAchtung: Überweise nie eine Kaution an Personen, deren Wohnung du nie gesehen hast oder weil sie gerade nicht in der Schweiz weilen.\t \t mehr SCHREIB MIR! Anzeige Crowdfunding Workshop Food ZH Aufstrebende Gastronomen aufgepasst! Am Crowdify Workshop erhalten ambitionierte Feinschmecker den Masterplan für eine erfolgreiche Crowdfunding-Aktion. Mehr Infos Werbeanzeige WG Folgen ♥ ♥ ♥ Unterstütze Ron ♥ ♥ ♥Ich bin dein Sprachrohr für deine Stadt und dabei stets unabhängig. Daher freue ich mich über alle, die bereit sind, einen kleinen, finanziellen Beitrag an mein Tun zu leisten. 0 auf Merkliste"

['C:\\coding\\wgscraper\\notebooks',
 'C:\\ProgramData\\Anaconda3\\envs\\wgscraper\\python38.zip',
 'C:\\ProgramData\\Anaconda3\\envs\\wgscraper\\DLLs',
 'C:\\ProgramData\\Anaconda3\\envs\\wgscraper\\lib',
 'C:\\ProgramData\\Anaconda3\\envs\\wgscraper',
 '',
 'C:\\ProgramData\\Anaconda3\\envs\\wgscraper\\lib\\site-packages',
 'C:\\ProgramData\\Anaconda3\\envs\\wgscraper\\lib\\site-packages\\win32',
 'C:\\ProgramData\\Anaconda3\\envs\\wgscraper\\lib\\site-packages\\win32\\lib',
 'C:\\ProgramData\\Anaconda3\\envs\\wgscraper\\lib\\site-packages\\Pythonwin',
 'C:\\ProgramData\\Anaconda3\\envs\\wgscraper\\lib\\site-packages\\IPython\\extensions',
 'C:\\Users\\cbole\\.ipython',
 WindowsPath('C:/coding/wgscraper/src'),
 WindowsPath('C:/coding/wgscraper/src'),
 'C:\\coding\\wgscraper\\src']

In [41]:

    
regex = {r'category1': r'Biete \/ Suche \/ Tausche: (.+?):',
         r'bid_ask': r'Biete \/ Suche \/ Tausche: .+?: (.+?) ',
         r'rent_buy': r'Mieten \& Kaufen: (.+?) ',
         r'rooms': r'Zimmer: ([^a-zA-Z ]+)',
         r'cost': r'Kosten: ([^a-zA-Z ]+)',
         r'address': r'Adresse: (.*?)Kontakt',
         r'duration': r'Vertragsart: (.*?) ',
         }
for i in [0,1]:
    d = {}
    for k, v in regex.items():
        prop = re.search(v, df.loc[i,'details'])
        if prop is not None:
            prop = prop.groups(0)[0]
        else:
            prop = 'unknown'
        print(prop)

Wohnung / Zimmer
Biete
Mieten
1
1'095.00
Marktgasse 19, 8038 Kloten 
unbefristet
Wohnung / Zimmer
Biete
Mieten
1.0
1'405.00
8003 Zürich 
unknown
